In [13]:
# default_exp qlearning.dqn_target

In [14]:
#export
import torch.nn.utils as nn_utils
from fastai.torch_basics import *
from fastai.data.all import *
from fastai.basics import *
from dataclasses import field,asdict
from typing import List,Any,Dict,Callable
from collections import deque
import gym
import torch.multiprocessing as mp
from torch.optim import *

from fastrl.data import *
from fastrl.async_data import *
from fastrl.basic_agents import *
from fastrl.learner import *
from fastrl.metrics import *
from fastrl.ptan_extension import *
from fastrl.qlearning.dqn import *

if IN_NOTEBOOK:
    from IPython import display
    import PIL.Image

# Target DQN

In [15]:
# export
class TargetDQNTrainer(Callback):
    def __init__(self,n_batch=0): store_attr()
    def after_pred(self):
        s,a,r,sp,d,er,steps=(self.learn.xb+self.learn.yb)
        exps=[ExperienceFirstLast(*o) for o in zip(*(self.learn.xb+self.learn.yb))]
        batch_targets=[calc_target(self.learn.model, exp.reward, exp.last_state,exp.done,self.learn.discount**self.learn.n_steps)
                         for exp in exps]
        

        state_action_values = self.learn.model(s.float()).gather(1, a.unsqueeze(-1)).squeeze(-1)
#         next_state_values = self.learn.target_model(sp.float()).max(1)[0]
        next_state_values=self.get_next_state_values(sp)
        next_state_values[d] = 0.0

        expected_state_action_values=next_state_values.detach()*(self.learn.discount**self.learn.n_steps)+r
#         print(*self.learn.yb,self.learn.pred)
#         print(self.learn.pred,self.learn.yb)
#         print(self.learn._yb,self.learn.yb[0])
        self.learn._yb=self.learn.yb
        self.learn.yb=(expected_state_action_values.float(),)
        self.learn.pred=state_action_values
        
    def get_next_state_values(self,sp):
        return self.learn.target_model(sp.float()).max(1)[0]
    
    def after_loss(self):
        self.learn.yb=self.learn._yb
        
    def after_batch(self):
        if self.n_batch%self.learn.target_sync==0:
            self.learn.target_model.load_state_dict(self.learn.model.state_dict())
        self.n_batch+=1

In [16]:
# export
class TargetDQNLearner(AgentLearner):
    def __init__(self,dls,discount=0.99,n_steps=3,target_sync=300,**kwargs):
        store_attr()
        self.target_q_v=[]
        super().__init__(dls,loss_func=nn.MSELoss(),**kwargs)
        self.target_model=deepcopy(self.model)

In [17]:
env='CartPole-v1'
model=LinearDQN((4,),2)
agent=DiscreteAgent(model=model.to(default_device()),device=default_device(),
                    a_selector=EpsilonGreedyActionSelector())

block=FirstLastExperienceBlock(agent=agent,seed=0,n_steps=3,dls_kwargs={'bs':32,'num_workers':0,'verbose':False,'indexed':True,'shuffle_train':False})
blk=IterableDataBlock(blocks=(block),
                      splitter=FuncSplitter(lambda x:False),
                     )
dls=blk.dataloaders([env]*1,n=32*1000,device=default_device())

learner=TargetDQNLearner(dls,agent=agent,n_steps=3,cbs=[EpsilonTracker,
                                        ExperienceReplay(sz=50000,bs=32,starting_els=32,max_steps=gym.make(env)._max_episode_steps),
                                        TargetDQNTrainer],metrics=[AvgEpisodeRewardMetric(experience_cls=ExperienceFirstLast)])
learner.fit(6,lr=0.001,wd=0)

epoch,train_loss,train_avg_episode_r,valid_loss,valid_avg_episode_r,time
0,4.006340,32.738333,None,32.738333,00:48
1,5.938926,69.090000,None,69.090000,00:49
2,3.645710,129.225000,None,129.225000,00:49
3,7.758656,202.300000,None,202.300000,00:48
4,2.646263,165.290000,None,165.290000,00:49
5,3.180485,175.050000,None,175.050000,00:47


In [1]:
# hide
from nbdev.export import *
from nbdev.export2html import *
notebook2script()
notebook2html()

Converted 00_core.ipynb.
Converted 01_wrappers.ipynb.
Converted 03_basic_agents.ipynb.
Converted 04_learner.ipynb.
Converted 05a_ptan_extend.ipynb.
Converted 05b_async_data.ipynb.
Converted 05c_data.ipynb.
Converted 13_metrics.ipynb.
Converted 14a_actorcritic.sac.ipynb.
Converted 14b_actorcritic.diayn.ipynb.
Converted 15_actorcritic.a3c_data.ipynb.
Converted 16_actorcritic.a2c.ipynb.
Converted 17_actorcritc.v1.dads.ipynb.
Converted 18_policy_gradient.ppo.ipynb.
Converted 19_policy_gradient.trpo.ipynb.
Converted 20a_qlearning.dqn.ipynb.
Converted 20b_qlearning.dqn_n_step.ipynb.
Converted 20c_qlearning.dqn_target.ipynb.
Converted 20d_qlearning.dqn_double.ipynb.
Converted 20e_qlearning.dqn_noisy.ipynb.
Converted index.ipynb.
Converted notes.ipynb.


converting: /opt/project/fastrl/nbs/14b_actorcritic.diayn.ipynb
converting: /opt/project/fastrl/nbs/13_metrics.ipynb
converting: /opt/project/fastrl/nbs/20d_qlearning.dqn_double.ipynb
converting: /opt/project/fastrl/nbs/20e_qlearning.dqn_noisy.ipynb
An error occurred while executing the following cell:
------------------
from nbdev.showdoc import show_doc
from fastrl.qlearning.dqn_double import *
------------------

---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
<ipython-input-1-f25b9eee321e> in <module>
      1 from nbdev.showdoc import show_doc
----> 2 from fastrl.qlearning.dqn_double import *

/opt/project/fastrl/fastrl/qlearning/dqn_double.py in <module>
     22 from ..ptan_extension import *
     23 from .dqn import *
---> 24 from .dqn_target import *
     25 
     26 if IN_NOTEBOOK:

/opt/project/fastrl/fastrl/qlearning/dqn_target.py in <module>
     89 from ..ptan_extension i